# 11. 标准库简介 —— 第二部分
第二部分涵盖了专业编程所需要的更高级的模块。这些模块很少用在小脚本中。

## 11.1. 格式化输出

In [ ]:
'''reprlib 模块提供了一个定制化版本的 repr() 函数，用于缩略显示大型或深层嵌套的容器对象:
'''
import reprlib
reprlib.repr(set('supercalifragilisticexpialidocious'))

In [ ]:
'''pprint 模块提供了更加复杂的打印控制，其输出的内置对象和用户自定义对象能够被解释器直接读取。
当输出结果过长而需要折行时，“美化输出机制”会添加换行符和缩进，以更清楚地展示数据结构:
'''
import pprint
t = [[[['black', 'cyan'], 'white', ['green', 'red']], [['magenta','yellow'], 'blue']]]
pprint.pprint(t, width=30)

In [ ]:
'''textwrap 模块能够格式化文本段落，以适应给定的屏幕宽度:
'''
import textwrap
doc = """The wrap() method is just like fill() except that it returns
a list of strings instead of one big string with newlines to separate
the wrapped lines."""
print(textwrap.fill(doc, width=40))

In [ ]:
'''locale 模块处理与特定地域文化相关的数据格式。locale 模块的 format 函数包含一个 grouping 属性，可直接将数字格式化为带有组分隔符的样式:
'''
import locale
locale.setlocale(locale.LC_ALL, 'English_United States.1252')

In [ ]:
conv = locale.localeconv()          # get a mapping of conventions
x = 1234567.8
locale.format_string("%d", x, grouping=True)

## 11.2. 模板

In [ ]:
'''string 模块包含一个通用的 Template 类，具有适用于最终用户的简化语法。它允许用户在不更改应用逻辑的情况下定制自己的应用。
'''
from string import Template
t = Template('${village}folk send $$10 to $cause.')
t.substitute(village='Nottingham', cause='the ditch fund')

In [ ]:
'''用户提供的数据有可能是不完整的，此时使用 safe_substitute() 方法更加合适 —— 如果数据缺失，它会直接将占位符原样保留。
'''
t = Template('Return the $item to $owner.')
d = dict(item='unladen swallow')
t.safe_substitute(d)

In [ ]:
'''Template 的子类可以自定义分隔符。
'''
import time, os.path
photofiles = ['img_1074.jpg', 'img_1076.jpg', 'img_1077.jpg']
class BatchRename(Template):
    delimiter = '%'

In [ ]:
fmt = input('Enter rename style (%d-date %n-seqnum %f-format):  ')

In [ ]:
t = BatchRename(fmt)
date = time.strftime('%d%b%y')
for i, filename in enumerate(photofiles):
    base, ext = os.path.splitext(filename)
    newname = t.substitute(d=date, n=i, f=ext)
    print('{0} --> {1}'.format(filename, newname))

## 11.3. 使用二进制数据记录格式

In [ ]:
import struct

with open('myfile.zip', 'rb') as f:
    data = f.read()

start = 0
for i in range(3):                      # show the first 3 file headers
    start += 14
    fields = struct.unpack('<IIIHH', data[start:start+16])
    crc32, comp_size, uncomp_size, filenamesize, extra_size = fields

    start += 16
    filename = data[start:start+filenamesize]
    start += filenamesize
    extra = data[start:start+extra_size]
    print(filename, hex(crc32), comp_size, uncomp_size)

    start += extra_size + comp_size     # skip to the next header

## 11.4. 多线程

In [ ]:
'''多线程应用面临的主要挑战是，相互协调的多个线程之间需要共享数据或其他资源。
为此，threading 模块提供了多个同步操作原语，包括线程锁、事件、条件变量和信号量。
应用程序使用 Queue 对象进行线程间通信和协调，更易于设计，更易读，更可靠。
'''
import threading, zipfile

class AsyncZip(threading.Thread):
    def __init__(self, infile, outfile):
        threading.Thread.__init__(self)
        self.infile = infile
        self.outfile = outfile

    def run(self):
        f = zipfile.ZipFile(self.outfile, 'w', zipfile.ZIP_DEFLATED)
        f.write(self.infile)
        f.close()
        print('Finished background zip of:', self.infile)

background = AsyncZip('mydata.txt', 'myarchive.zip')
background.start()
print('The main program continues to run in foreground.')

background.join()    # Wait for the background task to finish
print('Main program waited until background was done.')

## 11.5. 日志

In [ ]:
import logging
logging.debug('Debugging information')
logging.info('Informational message')
logging.warning('Warning:config file %s not found', 'server.conf')
logging.error('Error occurred')
logging.critical('Critical error -- shutting down')

## 11.6. 弱引用

In [ ]:
'''Python 会自动进行内存管理（对大多数对象进行引用计数并使用 garbage collection 来清除循环引用
当某个对象的最后一个引用被移除后不久就会释放其所占用的内存。
'''
import weakref, gc
class A:
    def __init__(self, value):
        self.value = value
    def __repr__(self):
        return str(self.value)
a = A(10)                   # create a reference
d = weakref.WeakValueDictionary()
d['primary'] = a            # does not create a reference
d['primary']                # fetch the object if it is still alive         

In [ ]:
del a  
gc.collect()   

In [ ]:
d['primary'] 

## 11.7. 用于操作列表的工具

In [ ]:
'''array 模块提供了一种 array() 对象，它类似于列表，但只能存储类型一致的数据且存储密集更高
'''
from array import array
a = array('H', [4000, 10, 700, 22222])
sum(a)
26932

In [ ]:
a[1:3]

In [ ]:
'''collections 模块提供了一种 deque() 对象，它类似于列表，但从左端添加和弹出的速度较快，而在中间查找的速度较慢。
此种对象适用于实现队列和广度优先树搜索:
'''
from collections import deque
d = deque(["task1", "task2", "task3"])
d.append("task4")
print("Handling", d.popleft())

In [ ]:
'''在替代的列表实现以外，标准库也提供了其他工具，例如 bisect 模块具有用于操作有序列表的函数:
'''
import bisect
scores = [(100, 'perl'), (200, 'tcl'), (400, 'lua'), (500, 'python')]
bisect.insort(scores, (300, 'ruby'))
scores

In [ ]:
'''heapq 模块提供了基于常规列表来实现堆的函数。 
'''
from heapq import heapify, heappop, heappush
data = [1, 3, 5, 7, 9, 2, 4, 6, 8, 0]
heapify(data)                      # rearrange the list into heap order
heappush(data, -5)                 # add a new entry
[heappop(data) for i in range(3)]  # fetch the three smallest entries

## 11.8. 十进制浮点运算

In [ ]:
'''decimal 模块提供了一种 Decimal 数据类型用于十进制浮点运算
'''
from decimal import *
round(Decimal('0.70') * Decimal('1.05'), 2)
Decimal('0.74')
round(.70 * 1.05, 2)

In [ ]:
Decimal('1.00') % Decimal('.10')

In [ ]:
1.00 % 0.10

In [ ]:
'''精确表示特性使得 Decimal 类能够执行对于二进制浮点数来说不适用的模运算和相等性检测:
'''
sum([Decimal('0.1')]*10) == Decimal('1.0')

In [ ]:
'''decimal 模块提供了运算所需要的足够精度:
'''
getcontext().prec = 36
Decimal(1) / Decimal(7)